In [ ]:
import pyvista as pv
import pvxarray
import xarray as xr
import numpy as np

In [ ]:
ds = xr.tutorial.load_dataset("air_temperature")
da = ds.air

algorithm = da.pyvista.algorithm(x="lon", y="lat", time="time")

In [ ]:
pl = pv.Plotter()
pl.add_mesh(algorithm)
pl.show(cpos="xy")

In [ ]:
algorithm.max_time_index

In [ ]:
ds = xr.tutorial.load_dataset("basin_mask")
da = ds.basin

# bottom, left, top, right
box = (-96.7030229414862, 90 - 64.98056667458044, -7.397202215623508, 90 + 23.180353571106195)

min_lat, min_lon, max_lat, max_lon = box

LatIndexer, LonIndexer = "Y", "X"
roi = da.loc[{LatIndexer: slice(min_lat, max_lat), LonIndexer: slice(min_lon, max_lon)}]

In [ ]:
roi

In [ ]:
mesh = roi.pyvista.mesh(x="X", y="Y", z="Z")

pl = pv.Plotter()
pl.add_mesh(mesh)
pl.set_scale(zscale=0.001)
pl.show()